## Template 101022


We need to load the following libraries. Make sure that the anndata version you have installed matches the one used by [cellxgene-schema](https://github.com/chanzuckerberg/single-cell-curation)

In [1]:
import pandas as pd
import scanpy as sc 
import anndata as ad
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
hdata = sc.read("To_Cellxgene_elo_lung_211122.h5ad")


In [ ]:
hdata = ad.read("To_Cellxgene_elo_lung_211122.h5ad")

In [ ]:
hdata.obs = hdata.obs.rename(columns={'Donor': 'donor_ID'})

In [ ]:
df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)


In [ ]:
hdata.obsp = adata.obsp

In [ ]:
adata

In [ ]:
pip show anndata

## Read data

Read the annotated and raw h5ad, or h5 file

In [ ]:
adata = sc.read("lung_5loc_sc_sn_cellxgene_23092022.h5ad")

In [ ]:
rdata = sc.read("lung_5loc_sc_sn_raw_cellxgene_23092022.h5ad")

## Prepare the hd5a layers need for CellxGene

### Prepare the obs layer

First we extract the obs layer from the h5ad file that has been provided to us. Then we extract out the obs layer as a csv file. 



In [ ]:
a_obs = adata.obs
a_obs.to_csv("a_obs_layer.csv")

In [ ]:
r_obs = rdata.obs
r_obs.to_csv("r_obs_layer.csv")

In [ ]:
a_obs

In [ ]:
r_obs

Download the excel spreadsheet from the INGEST data submission, and navigate to the sequence_file page.
Select the cell_suspension.biomaterial_core.biomaterial_id, cell_suspension.uuid, and library_preparation_protocol.protocol_core.protocol_id and filter for duplicates on those keys. This results in a unique set of cell_suspensions with their related data. 

See script here: https://github.com/ebi-ait/ingest-cellxgene-submitter#create-obs-layer-from-multiple-cell-suspension-uuids

The manual work here is  matching up unique HCA cell_suspensions and metadata with the  unique samples in the provided-h5ad file. We then generate a final obs_layer.csv which combines HCA metadata with the provided metadata from the contributor in matching rows. There is an opportunity for future automation here for scripts to perform the matching and to provide cell-type ontology terms. 

For a list of fields which should be in the final obs_layer.csv see (https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/3.0.0/schema.md)
- assay_ontology_term_id
- cell_type_ontology_term_id
- development_stage_ontology_term_id
- disease_ontology_term_id
- ethnicity_ontology_term_id
- is_primary_data
- organism_ontology_term_id
- sex_ontology_term_id
- tissue_ontology_term_id
- donor_id

There is also some manual work here to select cell-type ontology terms from the free text cell-type provided by the contributors. There is an opportunity for future automation here to get scripts / work with other experts in ontology to do this. 

Once we have the final combined obs_layer, then save it as "obs_layer.csv" and save it as a dobs2 object

In [ ]:
dobs2 = pd.read_csv("cellxgene_obs_layer.csv", sep=",", encoding= 'utf-8')

In [ ]:
dobs2.dtypes

In [ ]:
dobs2 = dobs2.set_index("Unnamed: 0", inplace = False)

Check here for redundant fields in the obs layer (e.g. tissue_label, donor_ext). Assay_ontology_term_id should be '10x v2 5' sequencing' as opposed to '10x TCR'.
Development_stage and tissue should be made specific (e.g. 35-year old human stage).
Unannotated cells use CL:0000003 (native cell).
Check that certain fields in the obs layer should not be continuous / numerical, but should be categorical or string. Use the cells below to fix this if necessary.

In [ ]:
adata.obs = dobs2

In [ ]:
adata.obs = adata.obs.drop(['Unnamed: 18','author_tissue','tissue_label','donor_ext'], axis = 1)

In [ ]:
dobs2.dtypes

In [ ]:
adata.obs = adata.obs.astype({'batch': 'category','cluster':'category'}, copy = False)

### Prepare the uns layer

In cases where the uns layer already exists, rather than creating the uns layer, we will add to it.
In this case, schema_version is 2.0.0 (although 3.0.0 is coming soon). Title is the name of the dataset. X_normalization is being deprecated in 3.0.0, but is the method of generating the X layer from the raw.X. 

In [ ]:
uns = adata.uns
uns.keys

In [ ]:
uns['schema_version']='3.0.0'
uns['title'] = "A spatial multi-omics atlas of the human lung reveals a novel immune cell survival niche"
uns

Else if the uns layer does not exist, then we will create it 

In [ ]:
uns ={
    "schema_version": "2.0.0",
    "title": "Transcriptomic characterisation of haematopoietic stem and progenitor cells from human adult bone marrow, spleen and peripheral blood",
    "X_normalization": "The 10x count matrices from both donors (after multiplet filtering) were processed with the Seurat (version 2.3.4) R package",
}

### Check what's in the obsm layer

There should be an obsm layer with spatial embeddings included. 

In [ ]:
adata.obsm

In [ ]:
obsm = adata.obsm

### Check what's in the X, var and obs layers before editing them

Check that the obs_names from both annotated and raw matrices are the same

In [ ]:
t = adata.obs_names==rdata.obs_names
np.count_nonzero(t)==adata.X.shape[0]

Assuming both obs layers do not match, not even in length, we would need to select from the raw layer those samples that are in the annotated layer. If they do match then ignore this. 

keep_rindices is a measure for the raw layer obs samples which are also in the annotated layer. 

In [ ]:
condition_rdata = rdata.obs_names.isin(adata.obs_names)
keep_rdata = rdata.obs_names[condition_rdata]
keep_rindices = np.where(condition_rdata == True)

### Prepare the var layer

In this case, we may need to perform different steps:
- Map the gene IDs to ENSEMBL IDs (using a specific GTF file)
- If any genes genes have been filtered out when the authors processed the raw matrix, they will not available in the annotated matrix. We need to add them to the processed matrix. We checked this by inspecting the matrices, but can be checked again by running the following cell

Check that both dataframes have the same number of rows. If they are different, the authors filtered out some genes from the PROCESSED, and we will need to add them in. 


In [ ]:
dvar = pd.DataFrame(data=adata.var)
rvar = pd.DataFrame(data=rdata.var)
dvar.shape[0] == rvar.shape[0]

In [ ]:
dvar.columns

In [ ]:
rvar.columns

Some of the features have been filtered out of the processed matrices. We have to add the filtered-out genes at the end of the matrices. For that, we are gonna first fill in the *feature_is_filtered* column at the rvar dataframe. We can then create a new dataframe dropping all the non filtered gene, and add this dataframe with the filtered genes at the end of dvar

In [ ]:
genes_add = [x for x in rdata.var.index.to_list() if x not in adata.var.index.to_list()]
all_genes = adata.var.index.to_list()
all_genes.extend(genes_add)
new_var = pd.DataFrame(index=all_genes)
new_var = pd.merge(new_var, rdata.var, left_index=True, right_index=True, how='left')
new_var['feature_is_filtered'] = False
new_var.loc[genes_add, 'feature_is_filtered'] = True

In [ ]:
dvar

In [ ]:
rvar

In [ ]:
rvar = rvar.set_index("gene_ids-1")

In [ ]:
dvar = dvar.set_index("gene_ids-1")

In [ ]:
dvar.pop('Gene_symbols')

To read in dvar from an external csv file (or if you want to edit it in Excel) use the following code

In [ ]:
dvar.to_csv("dvar.csv")

In [ ]:
dvar = pd.read_csv('dvar.csv')

### Prepare the X layer

In this case, we need to:
- Append to the processed X layer the counts for the genes that were filtered out. We will set them to 0.

#### Filter out the genes that did not map to ENSEMBL IDs

In [ ]:
if rdata.shape[1] > adata.shape[1]:
    genes_add = [x for x in rdata.var.index.to_list() if x not in adata.var.index.to_list()]
    new_matrix = csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape = rdata.shape)
    all_genes = adata.var.index.to_list()
    all_genes.extend(genes_add)
    new_var = pd.DataFrame(index=all_genes)
    new_var = pd.merge(new_var, rdata.var, left_index=True, right_index=True, how='left')
    new_var['feature_is_filtered'] = False
    new_var.loc[genes_add, 'feature_is_filtered'] = True
    new_adata = ad.AnnData(X=new_matrix, obs=adata.obs, var=dvar, uns=adata.uns, obsm=adata.obsm)
    new_adata = new_adata[:,rdata.var['gene_ids'].to_list()]
else:
    print('whoops')

In [ ]:
new_adata = new_adata[:,cxg_adata_raw.var.index.to_list()]


In [ ]:
print(new_adata.X)

In [ ]:
dExprs = normdata.X.toarray()
dExprs = pd.DataFrame(data=dExprs)
print(dExprs.shape)

In [ ]:
rExprs = rawdata.X.toarray()

In [ ]:
rExprs = rExprs.set_axis(rvar.index.to_list(), axis=1, inplace=False)

#### Append the genes filtered out by the authors when processing the X layer

In [ ]:
genesToAdd = dvar.loc[dvar['feature_is_filtered'] == True]

In [ ]:
dExprs = dExprs.reindex(columns=[*dExprs.columns.tolist(), *genesToAdd.index.to_list()], fill_value=0.0)

In [ ]:
dExprs

In [ ]:
dExprs.to_csv("dExprs_added_genes.csv")

In [ ]:
dExprs = dExprs.set_axis(dvar.index.to_list(), axis=1, inplace=False)
print(dExprs.shape)

## Prepare the hd5a file


Generate annotated h5ad matrix.
To speed up the matrices can also run: adata.X = csr_matrix(adata.X)



In [ ]:
# Generate annotated h5ad matrix
# To speed up the matrices can also run: adata.X = csr_matrix(adata.X)
hdata = ad.AnnData(X=adata.X, obs=dobs2, var=dvar ,obsm=obsm, uns=uns)

Save the annotated matrix

In [ ]:
new_adata.var.pop('mt')

In [ ]:
hdata.obs = hdata.obs.astype({'Age_in_years': 'category'})


In [ ]:
hdata.X

In [ ]:
hdata.X.shape

In [ ]:
adata

In [ ]:
adata.varm['PCs']

In [ ]:
adata.obsp['connectivities']

In [ ]:
hdata

In [ ]:
new_adata.obs['Age']

In [ ]:
new_adata.obs.columns

In [ ]:
new_adata.var.columns

In [ ]:
adata.write_h5ad("normalized_spacetime_matrix_edited.h5ad")

In [ ]:
hdata.obs.dtypes

In [ ]:
adata.obs.dtypes

In [ ]:
hdata.write_h5ad("normalized_spacetime_matrix_edited.h5ad")

Generate raw h5ad matrix, in this case raw.var is not the same as var, as some genes have been filtered out

In [ ]:
# Generate raw h5ad matrix, raw.var is not always going to be the same as var, but it is in this case because the
# genes being investigated are identical and nothing was filtered out 
idata = ad.AnnData(X=rdata.X, var=rvar)

In [ ]:
idata.write("raw_spacetime_matrix_edited.h5ad")

## Filter allowed ENSEMBL IDs
CellxGene only allows a set of genes (ENSEMBL IDs) to be included in their matrices. We need to filter those out.

In [ ]:
# Read in the saved objects if you start from here this section
hdata = sc.read('normalized_spacetime_matrix_edited.h5ad')
idata = sc.read('raw_spacetime_matrix_edited.h5ad')

In [ ]:
idata.var.drop("noVersionENS",axis=1,inplace=True)

In [ ]:
hdata.var['feature_is_filtered'] = False

In [ ]:
# idata.var.set_index("noVersionENS", inplace=True)
idata.var = idata.var.drop('feature_is_filtered', axis = 'columns')

In [ ]:
idata.var

In [ ]:
hdata.var.index

In [ ]:
#  Filter out allowed ENSEMBL IDs from annotated matrix
approved = pd.read_csv('/Users/wteh/Documents/Wrangling/Cellxgene/KnowledgeShare/allowed_genes.txt',dtype='str')['feature_id'].to_list()

var_to_keep = hdata.var.index.tolist()
var_to_keep
var_in_approved = hdata.var.index[hdata.var.index.isin(approved)].tolist()
var_to_keep = [e for e in var_to_keep if e in var_in_approved]
hdata = hdata[:, var_to_keep]

In [ ]:
var_to_keep = idata.var.index.tolist()
var_to_keep
var_in_approved = idata.var.index[idata.var.index.isin(approved)].tolist()
var_to_keep = [e for e in var_to_keep if e in var_in_approved]
idata = idata[:, var_to_keep]

In [ ]:
hdata.var.shape

In [ ]:
idata.var.shape

## Add raw matrix to hdata


In [ ]:
# Add raw X matrix and raw var to the h5ad matrix 
hdata.raw = idata

In [ ]:
#if needed, transfer to sparse matrix format
if type(hdata.X) != "sparse.csr.csr_matrix":
    print('converting X to sparse')
    hdata.X = csr_matrix(hdata.X)
# if hdata.raw:
#     if type(hdata.raw.X) != "sparse.csr.csr_matrix":
#         print('converting raw.X to sparse')
#         raw_adata = ad.AnnData(hdata.raw.X, var=hdata.raw.var, obs=hdata.obs)
#         raw_adata.X = csr_matrix(raw_adata.X)
#         hdata.raw = raw_adata

In [ ]:
hdata.write("To_Cellxgene_elo_lung_211122-3.h5ad")

In [ ]:
hdata.obs

In [ ]:
adata.obs.dtypes

## Validate before sending to CellxGene


Run the cellxgene-schema validate tool which validates if a h5ad object meets cellxgene schema. See details here: https://github.com/chanzuckerberg/single-cell-curation

In [ ]:
hdata = sc.read("To_Cellxgene_elo_lung_211122.h5ad")


In [ ]:
hdata.obs = dobs2

In [ ]:
hdata.obs

In [ ]:
hdata.uns = uns

In [ ]:
hdata.obs['disease_ontology_term_id']

In [ ]:
hdata.obs['disease_ontology_term_id'][hdata.obs['disease_ontology_term_id'] == 'EFO:0000384'] = 'MONDO:0005011'

In [ ]:
MONDO:0005011

In [ ]:
rawdata.var

In [ ]:
hdata.raw.var


In [ ]:
normdata.var

In [ ]:
hdata.var

In [ ]:
hdata.obs = hdata.obs.drop({'Age','Gender','Region code','sample name'}, axis = 'columns')

In [ ]:
hdata.var

In [ ]:
hda

In [ ]:
hdata.var['feature_biotype'] = 'gene'

In [ ]:
hdata.raw.var['feature_biotype'] = 'gene'

In [ ]:
hdata.obs = hdata.obs.astype({"disease_ontology_term_id":"category"})

In [ ]:
hdata.obs.dtypes

In [ ]:
hdata.var.pop('mt')

In [ ]:
hdata.obs['is_primary_data']

In [ ]:
hdata.obs['is_primary_data'] = hdata.obs['is_primary_data'].replace('no', False)


In [ ]:
hdata.obs = hdata.obs.astype({"is_primary_data":"bool"})

In [ ]:
hdata.obs['cell_type_ontology_term_id'][hdata.obs['cell_type_ontology_term_id'] == 'CL:0009111'] = 'CL:0000844'

In [ ]:
hdata.obs['cell_type_ontology_term_id']

In [ ]:
CL: 0009112, CL: 0009111, CL: 0009105

In [ ]:
CL:0009112 - centroblast - CL:0000844 (germinal center B cell)
CL:0009111 - centrocyte - CL:0000844 (germinal center B cell)
CL:0009105 - T cell zone reticular cell - CL:0009102 (lymph node fibroblastic reticular cell)

In [ ]:
rExprs = rExprs.set_axis(rvarOrIndex, axis=1, inplace=False)
rExprs = rExprs.loc[:,rKeepFeatIndex]
print(rExprs.shape)